In [18]:
import geopandas
import re
import pandas as pd
import folium
import requests
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
pd.options.mode.chained_assignment = None

In [19]:
# API Call Function

def call_api(url, lta_api_key):
    
    # Define endpoint to call
    endpoint = url

    # Define the headers, including the API key
    headers = {
        'accept': 'application/json',
        'AccountKey': lta_api_key
    }

    # Initialise the output list to hold the results
    output = []

    # Loop through the pages to get all the results
    while (len(output) % 500) == 0:

        # Set the params to skip to the correct page
        params = {"$skip": len(output)}

        # Make the GET request
        req = requests.get(endpoint, headers = headers, params = params)

        # Extend the output list to include the results
        output.extend(req.json()['value'])

    return pd.DataFrame(output)



In [20]:
# API Key

api_key = '15i2Wn8mTPuKL2qnVXqkJg=='

In [21]:
# Bus Services

bus_services = call_api("https://datamall2.mytransport.sg/ltaodataservice/BusServices", api_key)
bus_services

,ServiceNo,Operator,Direction,Category,OriginCode,DestinationCode,AM_Peak_Freq,AM_Offpeak_Freq,PM_Peak_Freq,PM_Offpeak_Freq,LoopDesc
0,118,GAS,1,TRUNK,65009,97009,5-08,8-12,8-10,09-14,
1,118,GAS,2,TRUNK,97009,65009,10-10,8-11,4-08,9-12,
2,118A,GAS,1,TRUNK,65009,96119,06-66,-,-,-,
3,118B,GAS,1,TRUNK,96111,65191,-,-,24-57,-,
4,119,GAS,1,TRUNK,65009,65009,09-13,12-18,12-15,15-17,Hougang St 21
...,...,...,...,...,...,...,...,...,...,...,...
722,98B,TTS,1,TRUNK,28501,21099,05-52,-,-,-,
723,98M,TTS,1,TRUNK,28009,28009,-,17-18,-,12-17,Corporation Rd
724,990,TTS,1,TRUNK,43009,43009,11-11,13-15,12-14,12-14,Jurong Gateway Rd
725,992,TTS,1,TRUNK,43009,41009,03-09,08-12,04-08,04-13,


In [22]:
# Bus Routes

bus_routes = call_api("https://datamall2.mytransport.sg/ltaodataservice/BusRoutes", api_key)
bus_routes

,ServiceNo,Operator,Direction,StopSequence,BusStopCode,Distance,WD_FirstBus,WD_LastBus,SAT_FirstBus,SAT_LastBus,SUN_FirstBus,SUN_LastBus
0,10,SBST,1,1,75009,0.0,0500,2300,0500,2300,0500,2300
1,10,SBST,1,2,76059,0.6,0502,2302,0502,2302,0502,2302
2,10,SBST,1,3,76069,1.1,0504,2304,0504,2304,0503,2304
3,10,SBST,1,4,96289,2.3,0508,2308,0508,2309,0507,2308
4,10,SBST,1,5,96109,2.7,0509,2310,0509,2311,0508,2309
...,...,...,...,...,...,...,...,...,...,...,...,...
25543,9B,SBST,1,25,95091,9.5,0741,0817,-,-,-,-
25544,9B,SBST,1,26,95131,9.7,0742,0818,-,-,-,-
25545,9B,SBST,1,27,95141,10.2,0744,0820,-,-,-,-
25546,9B,SBST,1,28,95061,10.6,0745,0821,-,-,-,-


In [23]:
# Bus Stops

bus_stops = call_api("https://datamall2.mytransport.sg/ltaodataservice/BusStops", api_key)
bus_stops

,BusStopCode,RoadName,Description,Latitude,Longitude
0,01012,Victoria St,Hotel Grand Pacific,1.296848,103.852536
1,01013,Victoria St,St. Joseph's Ch,1.297710,103.853225
2,01019,Victoria St,Bras Basah Cplx,1.296990,103.853022
3,01029,Nth Bridge Rd,Opp Natl Lib,1.296673,103.854414
4,01039,Nth Bridge Rd,Bugis Cube,1.298208,103.855491
...,...,...,...,...,...
5135,99139,Changi Village Rd,Blk 5,1.388195,103.987234
5136,99161,Nicoll Dr,Aft Changi Beach CP 3,1.390262,103.992957
5137,99171,Nicoll Dr,Changi Beach CP 2,1.391128,103.991021
5138,99181,Telok Paku Rd,Bef S'pore Aviation Ac,1.387754,103.988503


In [24]:
# Bus Routes with Bus Stop Descriptions

bus_route_names = pd.merge(bus_routes, bus_stops, on = 'BusStopCode', how = 'left')
bus_route_names

,ServiceNo,Operator,Direction,StopSequence,BusStopCode,Distance,WD_FirstBus,WD_LastBus,SAT_FirstBus,SAT_LastBus,SUN_FirstBus,SUN_LastBus,RoadName,Description,Latitude,Longitude
0,10,SBST,1,1,75009,0.0,0500,2300,0500,2300,0500,2300,Tampines Ctrl 1,Tampines Int,1.354076,103.943391
1,10,SBST,1,2,76059,0.6,0502,2302,0502,2302,0502,2302,Tampines Ave 5,Opp Our Tampines Hub,1.352962,103.941652
2,10,SBST,1,3,76069,1.1,0504,2304,0504,2304,0503,2304,Tampines Ave 5,Blk 147,1.348753,103.942086
3,10,SBST,1,4,96289,2.3,0508,2308,0508,2309,0507,2308,Simei Ave,Changi General Hosp,1.340055,103.948381
4,10,SBST,1,5,96109,2.7,0509,2310,0509,2311,0508,2309,Simei Ave,Opp Blk 3012,1.337371,103.950673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25543,9B,SBST,1,25,95091,9.5,0741,0817,-,-,-,-,Nicoll Dr,SAF Ferry Ter,1.387214,103.999714
25544,9B,SBST,1,26,95131,9.7,0742,0818,-,-,-,-,Airline Rd,Police Pass Off,1.385252,103.999834
25545,9B,SBST,1,27,95141,10.2,0744,0820,-,-,-,-,Airline Rd,Aft Cargo Bldg D,1.381386,103.998191
25546,9B,SBST,1,28,95061,10.6,0745,0821,-,-,-,-,Airline Rd,Airline Hse,1.377552,103.996539


In [123]:
bus_stop_67 = bus_route_names[(bus_route_names["ServiceNo"] == '67') & (bus_route_names["Direction"] == 2)]
bus_stop_67

#bus_stop_codes = bus_stop_67["BusStopCode"]
#bus_stop_codes

,ServiceNo,Operator,Direction,StopSequence,BusStopCode,Distance,WD_FirstBus,WD_LastBus,SAT_FirstBus,SAT_LastBus,SUN_FirstBus,SUN_LastBus,RoadName,Description,Latitude,Longitude
17352,67,SMRT,2,1,75009,0.0,0530,2330,0530,2330,0530,2330,Tampines Ctrl 1,Tampines Int,1.354076,103.943391
17353,67,SMRT,2,2,76059,0.6,0532,2332,0532,2332,0532,2332,Tampines Ave 5,Opp Our Tampines Hub,1.352962,103.941652
17354,67,SMRT,2,3,76069,1.1,0533,2333,0534,2334,0534,2334,Tampines Ave 5,Blk 147,1.348753,103.942086
17355,67,SMRT,2,4,75059,1.8,0536,2336,0537,2337,0537,2337,Tampines Ave 1,BEF TAMPINES WEST STN,1.344712,103.940217
17356,67,SMRT,2,5,75069,2.5,0538,2338,0539,2338,0539,2339,Bedok Reservoir Rd,Blk 960A,1.340663,103.936700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17431,67,SMRT,2,80,44129,31.3,0603,0057,0609,0100,0602,0100,Choa Chu Kang Way,Opp Blk 113,1.377417,103.750927
17432,67,SMRT,2,81,44449,31.7,0605,0058,0611,0102,0604,0102,Choa Chu Kang Way,Palm Gdns,1.378892,103.747627
17433,67,SMRT,2,82,44459,32.1,0607,0059,0613,0104,0606,0104,Choa Chu Kang Way,Opp South View Stn,1.380287,103.744546
17434,67,SMRT,2,83,44469,32.4,0609,0101,0615,0106,0608,0106,Choa Chu Kang Way,Blk 414,1.382439,103.742752


In [25]:
# Bus Stop Codes & Coordinates

bus_stops_coordinates = bus_stops[['BusStopCode', 'Latitude', 'Longitude']]
bus_stops_coordinates['BusStopCode'] = bus_stops_coordinates['BusStopCode'].astype(int)

bus_stops_coordinates

,BusStopCode,Latitude,Longitude
0,1012,1.296848,103.852536
1,1013,1.297710,103.853225
2,1019,1.296990,103.853022
3,1029,1.296673,103.854414
4,1039,1.298208,103.855491
...,...,...,...
5135,99139,1.388195,103.987234
5136,99161,1.390262,103.992957
5137,99171,1.391128,103.991021
5138,99181,1.387754,103.988503


# Bus Passenger Volume

In [12]:
"""
# Passenger Volume by Origin Destination Bus Stop
bus_volume = call_api("https://datamall2.mytransport.sg/ltaodataservice/PV/ODBus", api_key)
print(bus_volume['Link'].tolist())
"""

['https://ltafarecard.s3.ap-southeast-1.amazonaws.com/202409/origin_destination_bus_202409.zip?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEFkaDmFwLXNvdXRoZWFzdC0xIkcwRQIgH3%2F%2FuUczvTzCAco8KU1PxKgeo9%2Fq0iYfOIOBO%2FpJI%2B4CIQCE5M2Rgnx4%2FOAc8Py3H37vqkdfFpKWtvsWqm1aINXuRirLBQjS%2F%2F%2F%2F%2F%2F%2F%2F%2F%2F8BEAQaDDM0MDY0NTM4MTMwNCIMxMrGyjQOdMDkFVFXKp8FMtgKSnhWwsqV%2BHaHlDvGTGyRM8M4KScYHU0UBvapfDrU9zrsaAM7u5h3YqZWeL1nSnRzvb0Ohbyripjn8oz2C2IHfEnBUzq026TZBAKeHII7hIBjUj9peb%2FVHooUllpWB3UNbUK3PHKmB5socTu0CsCD%2BCsT4EKRAqqV2jVDr03hHEjB81wauV8Gb5xnAXwLbhkezwWY%2BjjmR7uZaZ398KMX3agqfxxCBi2pi%2B8uoexGzmbWJd1lkuUGVvliQh4khQgrxTkMOdgnIF7yvqVjTXdK00v4V%2Fd1PgSWPZ0%2B%2B1iW7PtNDNa%2B2gMl8oY7lIuRbAnrnVvWH8OQiouYLLfC4sN1K5CuZafw4Uf8Rz%2FM7v%2FMa3RAOubTb%2B2zAsuJdWzIMDwASa7EbqLQqgCoy64ha%2Bl4olscGZgIUNXEht%2BHczIPJ4j14VQAROyVSxxMpKmh7mYdvNiqTYNbnqVNeSQXquEUIwmhzKRjXPUYjYJ5Po4D6eEZK%2BKGnSmWMUmUsPI6jhfKcSegH%2By2%2BAgv6D6cwBtux1CbX0l86ZU8qRDixofGrx0MDVMBL8qjfGaM27Igz%2Ba2YuW9XgGKmOIxs3B81%2F4TFPWRem6iPDeZyc

In [ ]:
"""
bus_volume = pd.read_csv('./origin_destination_bus_202409.csv')
bus_volume
"""

In [16]:
"""
# Passenger Volume by Origin Destination Bus Stops with Bus Stop Coordinates

bus_volume_coordinates = pd.merge(bus_volume, bus_stops_coordinates, left_on = 'ORIGIN_PT_CODE', right_on = 'BusStopCode', how = 'left')
bus_volume_coordinates.rename(columns = {'Latitude': 'Origin Latitude', 'Longitude': 'Origin Longitude'}, inplace = True)

bus_volume_coordinates = pd.merge(bus_volume_coordinates, bus_stops_coordinates, left_on = 'DESTINATION_PT_CODE', right_on = 'BusStopCode', how = 'left')
bus_volume_coordinates.rename(columns = {'Latitude': 'Destination Latitude', 'Longitude': 'Destination Longitude'}, inplace = True)

bus_volume_coordinates

"""

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS,BusStopCode_x,Origin Latitude,Origin Longitude,BusStopCode_y,Destination Latitude,Destination Longitude
0,2024-09,WEEKENDS/HOLIDAY,20,BUS,46009,46249,586,46009,1.436946,103.785936,46249,1.442112,103.778748
1,2024-09,WEEKENDS/HOLIDAY,19,BUS,59091,59419,54,59091,1.431652,103.835108,59419,1.425360,103.846070
2,2024-09,WEEKENDS/HOLIDAY,23,BUS,54181,54201,72,54181,1.370533,103.836535,54201,1.375969,103.839271
3,2024-09,WEEKENDS/HOLIDAY,14,BUS,61039,61111,256,61039,1.338453,103.870849,61111,1.335500,103.872035
4,2024-09,WEEKENDS/HOLIDAY,14,BUS,44629,44009,902,44629,1.378512,103.736449,44009,1.385869,103.745789
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5721498,2024-09,WEEKENDS/HOLIDAY,8,BUS,53059,53181,1,53059,1.355471,103.831223,53181,1.356073,103.837810
5721499,2024-09,WEEKENDS/HOLIDAY,13,BUS,70071,94051,1,70071,1.333772,103.898413,94051,1.317102,103.947380
5721500,2024-09,WEEKENDS/HOLIDAY,20,BUS,8069,22481,1,8069,1.298214,103.849361,22481,1.342462,103.705541
5721501,2024-09,WEEKENDS/HOLIDAY,0,BUS,44809,44649,1,44809,1.385001,103.759784,44649,1.382070,103.762963


In [ ]:
#Passenger Volume by Origin Destination Train Stations

# train_volume = call_api("https://datamall2.mytransport.sg/ltaodataservice/PV/ODTrain", api_key)
# print(train_volume['Link'].tolist())

# train_volume = pd.read_csv('origin_destination_train_202409.csv')
# train_volume

['https://ltafarecard.s3.ap-southeast-1.amazonaws.com/202409/origin_destination_train_202409.zip?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEP7%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaDmFwLXNvdXRoZWFzdC0xIkcwRQIhAOrFHmo9GbM%2B387oC98gLva6LLZ5O50056xti6SiMllgAiBM3MnZazBXnZG81o%2BzJFajHNVIDSGOLHwwPL0B3R%2BZsirCBQh3EAQaDDM0MDY0NTM4MTMwNCIMU3DeWPzjdPo6SIlsKp8FCo7j%2FlX2uJYqIWjYMvDHfddWsDM1fqFFs6wMh6Itf7pJD9WQbkU6kMmdxXzyZ8w7%2BF6nwDlgcZDQQMVhok4%2BOcnrJTKM9k5T9jpjPlcn8hQAmJ5UantYHKSwl3HJTUis07U%2FTmn8Lj%2F7CXor5QvKHZvoYpZLYNnUW1nAbr%2FA3DZA3%2FvJnWi2F4ZIEynEt5tf2u6YhZb8AX8paLMDTDhl%2Fle6dgXmMCOBd%2FXjoOsn%2FU3xyqSNEazO%2F2orWg8sUjDJ9OMBMKgrL%2FcBuLo0yQAWWtruMCv8mcJjmOG3cmDEQ7kq6fF1r1TbvxFHMi8F0peo%2Br41MNj1prHpO%2B2vPgtUrq%2FMqW2o8CIBegpdnV1dl8vQOsUCfuC6uhLGIF%2Bryjq5oOXNngsxZaBB96POidkhFAoEcBKt9YawwNVlAsvxLuKjWJ0%2BYmgLax3%2F13FREyllRezfu3o676PQe%2BiEXU3wve1Mj1b0k6VR6jKMzVcyYDtCVBDVZTR1ow9e3irHdWKAl8GHMUQX%2FJMR6%2F14tr36rRSsI%2BJ8UwXgnE95mHCpR1NS9u7mliPcC%2BUsZD5EUb0Uio2LyBdylIO4pPnrkN8VfUzpP6lSz6nzha

In [ ]:
#passenger_volume_bus = call_api("https://datamall2.mytransport.sg/ltaodataservice/PV/Bus", api_key)
#print(passenger_volume_bus['Link'].tolist())

In [224]:
'''
# Adjusted API Call Function for Specific Date
def call_api_bus(url, lta_api_key, date):
    
    # Define the headers, including the API key
    headers = {
        'accept': 'application/json',
        'AccountKey': lta_api_key
    }

    # Initialize the output list to hold the results
    output = []

    # Loop through the pages to get all the results for the specified date
    while (len(output) % 500) == 0:
        
        # Set the params to skip to the correct page and include the date
        params = {
            "$skip": len(output),
            "Date": date
        }

        # Make the GET request
        req = requests.get(url, headers=headers, params=params)
        
        # Check for successful request and break if there's no data left to fetch
        if req.status_code != 200 or 'value' not in req.json():
            break

        # Extend the output list to include the results
        output.extend(req.json()['value'])

    return pd.DataFrame(output)


'''


In [ ]:
'''
bus_url = 'https://datamall2.mytransport.sg/ltaodataservice/PV/Bus'
date = '202408' 

bus_data = call_api_bus(bus_url, api_key, date)
print(bus_data['Link'].tolist())

'''